In [1]:
from arff_parser import *
from naive_bayes import *
import math
import copy

In [2]:
def mutual_information(list1, fea1, col1, list2, fea2, col2, list3, fea3, col3, data):
    cnt = 0
    denom = 0
    fea1_cnt = 0
    fea2_cnt = 0
    for row in data:
        if list1[fea1] == row[col1] and list2[fea2] == row[col2] and list3[fea3] == row[col3]:
            cnt += 1
        if list3[fea3] == row[col3]:
            denom += 1
        if list1[fea1] == row[col1] and list3[fea3] == row[col3]:
            fea1_cnt += 1
        if list2[fea2] == row[col2] and list3[fea3] == row[col3]:
            fea2_cnt += 1
    # p(xi, xj, y)
    a = (cnt+1)/(len(data) + len(list1)*len(list2)*len(list3))
    # p(xi, xj | y)
    b = (cnt+1)/(denom + len(list1)*len(list2))
    # p(xi | y)
    c = (fea1_cnt+1)/(denom + len(list1))
    # p(xj | y)
    d = (fea2_cnt+1)/(denom + len(list2))
    return a*math.log2(b/(c*d))

In [3]:
a = arff_data("mushroom_train.arff")
mutual_information(a.attributes[0].attribute_list, 3, 0, a.attributes[2].attribute_list, 0, 2, a.all_attributes[-1].attribute_list, 1, -1, a.data)

-0.0014046415078992638

In [4]:
# returns an adjacency matrix graph
def get_mutual_information(arffdata):
    mutual_info = []
    for i in range(len(arffdata.attributes)):
        temp_list = []
        for j in range(0, len(arffdata.attributes)):
            temp = 0
            if i == j:
                temp_list.append(-1)
                continue
            for y in range(len(arffdata.label.attribute_list)):
                for xi in range(len(arffdata.attributes[i].attribute_list)):
                    for xj in range(len(arffdata.attributes[j].attribute_list)):
                        temp += mutual_information(arffdata.attributes[i].attribute_list, xi, i, arffdata.attributes[j].attribute_list, xj, j, arffdata.label.attribute_list, y, -1, arffdata.data)
            temp_list.append(temp)
        mutual_info.append(temp_list)
    return mutual_info

In [5]:
mu = get_mutual_information(a)
len(mu)
mu

[[-1,
  0.05434753005360106,
  0.21740468685864836,
  0.02094890374472428,
  0.2610409953597202,
  0.09780359137473219,
  0.05316208365090882,
  0.02650572756960061,
  0.23905189438680852,
  0.02982937601367495,
  0.03286593092322269,
  0.08066359611939565,
  0.07955975553336177,
  0.1940182845787756,
  0.19617781688588812,
  0.033032845494039925,
  0.09780359137473219,
  0.06373174263627567,
  0.20211116713205776,
  0.2160656163450765,
  0.1031947084224693,
  0.16121591187740333],
 [0.05434753005360105,
  -1,
  0.15836974539107238,
  0.10697264037600634,
  0.1781811795907506,
  0.0348615747453022,
  0.1302874783123394,
  0.08714067784446189,
  0.1558348437542396,
  0.01587085035726145,
  0.01536559367246431,
  0.15347094488198165,
  0.14933229956981314,
  0.17468560086335191,
  0.1755673930049306,
  0.011729161143655216,
  0.0348615747453022,
  0.02235556746860399,
  0.24465219554942388,
  0.15426476999032368,
  0.1642670452953881,
  0.17599693902512314],
 [0.21740468685864836,
  0.15

In [6]:
class Graph:
    def __init__(self):
        self.edges = []
    def add_edge(self, col1, col2):
        self.edges.append([col1, col2])

In [7]:
def find_max_index(m):
    maximum = -100000000
    max_index = [-1, -1]
    for i in range(len(m)):
        for j in range(len(m[i])):
            if m[i][j] > maximum:
                maximum = m[i][j]
                max_index = [i, j]
    return max_index[0], max_index[1]

In [8]:
def find_max_in_row(row):
    maximum = -100000000
    max_index = -1
    for i in range(len(row)):
        if row[i] > maximum:
            maximum = row[i]
            max_index = i
    return max_index, maximum

In [9]:
b = [1,2,3,7,1,2,100,2]
find_max_in_row(b)

(6, 100)

In [10]:
m = [[1,2,300], [10,2,1], [100,200,2]]
find_max_index(m)

(0, 2)

In [11]:
def prims_algorithm(matrix, arffdata):
    # make a copy
    m = copy.deepcopy(matrix)
    V = list(range(len(arffdata.attributes)))
    Vnew = [0]
    G = Graph()
    
    while set(Vnew) != set(V):
        max_weight = -100000000
        max_x_index = -1
        max_y_index = -1
        for i in Vnew:
            temp_index, temp_weight = find_max_in_row(m[i])
            if temp_weight > max_weight:
                    max_weight = temp_weight
                    max_x_index = i
                    max_y_index = temp_index
        if max_y_index not in Vnew:
            m[max_x_index][max_y_index] = -1
            Vnew.append(max_y_index)
            G.add_edge(max_x_index, max_y_index)
        else:
            m[max_x_index][max_y_index] = -1

    return G

In [12]:
G = prims_algorithm(mu, a)

In [13]:
G.edges

[[0, 4],
 [4, 19],
 [4, 18],
 [18, 14],
 [18, 13],
 [18, 8],
 [4, 2],
 [18, 21],
 [18, 20],
 [18, 12],
 [18, 11],
 [18, 6],
 [4, 7],
 [18, 1],
 [21, 3],
 [8, 5],
 [8, 16],
 [8, 17],
 [4, 9],
 [8, 15],
 [8, 10]]

In [14]:
class Node:
    def __init__(self, value):
        self.value = value
        self.parents = []
    def add_parent(self, parent):
        self.append(parent)
    def get_parents_from_graph(self, G):
        for i in G.edges:
            if i[-1] == self.value:
                self.parents.append(i[0])

In [15]:
n = Node(4)
n.get_parents_from_graph(G)

In [16]:
def TAN_learning(arffdata, G):
    
    # all p(Xi | C, Xparent)
    all_probs = []
    for y in arffdata.label.attribute_list:
        probabilities = []
        for i in range(len(arffdata.attributes)):
            # doesn't include the root
            if i == 0:
                probabilities.append([])
                continue
            n = Node(i)
            n.get_parents_from_graph(G)
            parent = n.parents[0]
            prob_list = []
            #print(arffdata.attributes[i].attribute_list, arffdata.attributes[parent].attribute_list)
            for xi in arffdata.attributes[i].attribute_list:
                temp_list = []
                for xj in arffdata.attributes[parent].attribute_list:
                    cnt = 0
                    cnt2 = 0
                    for row in arffdata.data:
                        if row[i] == xi and row[parent] == xj and row[-1] == y:
                             cnt += 1
                        if row[parent] == xj and row[-1] == y:
                             cnt2 += 1
                    prob = (cnt+1)/(cnt2+len(arffdata.attributes[i].attribute_list))
                    temp_list.append(prob)
                print(i, parent, temp_list)
                prob_list.append(temp_list)
            probabilities.append(prob_list)
        all_probs.append(probabilities)
        
    # p(Xroot | C)
    y_list = []
    for y in arffdata.label.attribute_list:
        x_list = []
        for x in arffdata.attributes[0].attribute_list:
            pcnt = 0
            ycnt = 0
            for row in arffdata.data:
                if row[0] == x and row[-1] == y:
                    pcnt += 1
                if row[-1] == y:
                    ycnt += 1
            x_list.append((pcnt+1)/(ycnt + len(arffdata.attributes[0].attribute_list)))
        y_list.append(x_list)
            
    return all_probs, y_list

In [17]:
prob, y = TAN_learning(a, G)

1 18 [0.25, 0.5189873417721519, 0.2, 0.25, 0.25, 0.46595744680851064, 0.25, 0.25]
1 18 [0.25, 0.012658227848101266, 0.2, 0.25, 0.25, 0.0007092198581560284, 0.25, 0.25]
1 18 [0.25, 0.02531645569620253, 0.4, 0.25, 0.25, 0.5297872340425532, 0.25, 0.25]
1 18 [0.25, 0.4430379746835443, 0.2, 0.25, 0.25, 0.0035460992907801418, 0.25, 0.25]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.3054619015509103, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.0013486176668914363, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.0006743088334457181, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.3425488873904248, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.0006743088334457181, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.0006743088334457181, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.0013486176668914363, 0.1, 0.1]
2 4 [0.0625, 0.07692307692307693, 0.1, 0.1, 0.1, 0.1, 0.3290627

21 18 [0.14285714285714285, 0.012195121951219513, 0.125, 0.14285714285714285, 0.14285714285714285, 0.0007077140835102619, 0.14285714285714285, 0.14285714285714285]
21 18 [0.14285714285714285, 0.012195121951219513, 0.125, 0.14285714285714285, 0.14285714285714285, 0.004246284501061571, 0.14285714285714285, 0.14285714285714285]
21 18 [0.14285714285714285, 0.012195121951219513, 0.125, 0.14285714285714285, 0.14285714285714285, 0.0014154281670205238, 0.14285714285714285, 0.14285714285714285]
21 18 [0.14285714285714285, 0.012195121951219513, 0.125, 0.14285714285714285, 0.14285714285714285, 0.0007077140835102619, 0.14285714285714285, 0.14285714285714285]
21 18 [0.14285714285714285, 0.024390243902439025, 0.125, 0.14285714285714285, 0.14285714285714285, 0.0007077140835102619, 0.14285714285714285, 0.14285714285714285]
21 18 [0.14285714285714285, 0.012195121951219513, 0.25, 0.14285714285714285, 0.14285714285714285, 0.9907997169143666, 0.14285714285714285, 0.14285714285714285]
1 18 [0.25, 0.2, 0.25

14 18 [0.1111111111111111, 0.1, 0.1111111111111111, 0.34146341463414637, 0.1111111111111111, 0.004830917874396135, 0.1111111111111111, 0.1111111111111111]
14 18 [0.1111111111111111, 0.1, 0.1111111111111111, 0.003048780487804878, 0.1111111111111111, 0.004830917874396135, 0.1111111111111111, 0.1111111111111111]
14 18 [0.1111111111111111, 0.1, 0.1111111111111111, 0.003048780487804878, 0.1111111111111111, 0.004830917874396135, 0.1111111111111111, 0.1111111111111111]
14 18 [0.1111111111111111, 0.1, 0.1111111111111111, 0.003048780487804878, 0.1111111111111111, 0.004830917874396135, 0.1111111111111111, 0.1111111111111111]
14 18 [0.1111111111111111, 0.1, 0.1111111111111111, 0.3048780487804878, 0.1111111111111111, 0.004830917874396135, 0.1111111111111111, 0.1111111111111111]
14 18 [0.1111111111111111, 0.1, 0.1111111111111111, 0.003048780487804878, 0.1111111111111111, 0.004830917874396135, 0.1111111111111111, 0.1111111111111111]
14 18 [0.1111111111111111, 0.2, 0.1111111111111111, 0.0030487804878

In [18]:
prob[0][1][0][0]

0.25

In [19]:
prob[1][2][1][6]

0.1

In [20]:
y[1][3]

0.799618320610687

In [21]:
def TAN_inference(row, mutual_prob, root_prob, prior, G, arffdata):
    feature_index = label_to_index(row, arffdata)
    nom_prob0 = 1
    nom_prob1 = 1
    for i in range(1, len(row)-1):
        n = Node(i)
        n.get_parents_from_graph(G)
        parent = n.parents[0]
        nom_prob0 *= mutual_prob[0][i][feature_index[i]][feature_index[parent]]
        nom_prob1 *= mutual_prob[1][i][feature_index[i]][feature_index[parent]]
    nom_prob0 *= root_prob[0][feature_index[0]]*prior[0]
    nom_prob1 *= root_prob[1][feature_index[0]]*prior[1]
    
    prob1 = nom_prob0/(nom_prob0 + nom_prob1)
    prob2 = nom_prob1/(nom_prob0 + nom_prob1)
    return prob1, prob2

In [22]:
t = arff_data("mushroom_test.arff")
prior = calculate_prior(a.label.attribute_list, a.data)
TAN_inference(t.data[3], prob, y, prior, G, a)

(2.4864283127848175e-11, 0.9999999999751358)

In [23]:
prob[0][8]

[[0.08333333333333333,
  0.25287356321839083,
  0.07692307692307693,
  0.08333333333333333,
  0.08333333333333333,
  0.0028208744710860366,
  0.08333333333333333,
  0.08333333333333333],
 [0.08333333333333333,
  0.2413793103448276,
  0.07692307692307693,
  0.08333333333333333,
  0.08333333333333333,
  0.2461212976022567,
  0.08333333333333333,
  0.08333333333333333],
 [0.08333333333333333,
  0.011494252873563218,
  0.07692307692307693,
  0.08333333333333333,
  0.08333333333333333,
  0.0007052186177715092,
  0.08333333333333333,
  0.08333333333333333],
 [0.08333333333333333,
  0.19540229885057472,
  0.15384615384615385,
  0.08333333333333333,
  0.08333333333333333,
  0.0007052186177715092,
  0.08333333333333333,
  0.08333333333333333],
 [0.08333333333333333,
  0.011494252873563218,
  0.07692307692307693,
  0.08333333333333333,
  0.08333333333333333,
  0.0014104372355430183,
  0.08333333333333333,
  0.08333333333333333],
 [0.08333333333333333,
  0.011494252873563218,
  0.0769230769230769

In [24]:
for row in t.data:
    print(TAN_inference(row, prob, y, prior, G, a))

(0.096651045309998, 0.903348954690002)
(3.342315588673702e-07, 0.9999996657684411)
(0.9907486931185779, 0.009251306881422098)
(2.4864283127848175e-11, 0.9999999999751358)
(2.928770201331527e-09, 0.9999999970712298)
(5.1051852982378695e-12, 0.9999999999948949)
(3.1897319332562156e-12, 0.9999999999968102)
(5.318609069795584e-11, 0.9999999999468139)
(1.93935697781491e-08, 0.9999999806064302)
(2.2562098498886474e-13, 0.9999999999997744)
(6.81484898591057e-08, 0.9999999318515101)
(7.34261760584188e-10, 0.9999999992657383)
(5.3951029891466925e-09, 0.999999994604897)
(0.0017517070117267514, 0.9982482929882732)
(8.822879650704863e-07, 0.999999117712035)
(7.477979888260014e-14, 0.9999999999999252)
(5.318609069795584e-11, 0.9999999999468139)
(3.6740391467530042e-09, 0.9999999963259609)
(0.9895842914778403, 0.010415708522159636)
(1.6906916697416932e-08, 0.9999999830930832)
(1.999758337413559e-14, 0.99999999999998)
(0.0005734039889273122, 0.9994265960110728)
(9.783741932887572e-08, 0.9999999021625

(1.7860014617130374e-09, 0.9999999982139985)
(5.145919956995564e-11, 0.9999999999485408)
(1.7574709317004529e-06, 0.9999982425290682)
(3.715354071103422e-08, 0.9999999628464593)
(0.0017826966005995526, 0.9982173033994004)
(8.200489339168952e-06, 0.9999917995106609)
(2.4876403949308917e-12, 0.9999999999975122)
(1.148564180518552e-07, 0.999999885143582)
(1.2426429564626077e-05, 0.9999875735704353)
(0.0011347399064850225, 0.998865260093515)
(0.018089666889856908, 0.9819103331101431)
(4.541246040021596e-09, 0.9999999954587538)
(4.424451563878799e-09, 0.9999999955755484)
(1.0405968737732535e-09, 0.9999999989594031)
(0.8822065291833583, 0.11779347081664171)
(1.3996339658081623e-12, 0.9999999999986003)
(5.501268368448697e-10, 0.9999999994498732)
(0.0045126483418934295, 0.9954873516581065)
(0.0018182127478546112, 0.9981817872521453)
(0.9454950507935508, 0.054504949206449146)
(8.338011023411663e-07, 0.9999991661988977)
(0.06355068217486329, 0.9364493178251367)
(2.6243136858870898e-12, 0.9999999